In [9]:
# imports
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,log_loss,r2_score
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer,make_column_selector,make_column_transformer
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier

import os
os.cpu_count()

16

In [10]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 

In [11]:
y.head()

,quality
0,5
1,5
2,5
3,6
4,5


In [12]:
# split the data
X_train,  X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, stratify=y,random_state=25)

In [13]:
# scaling
scaler=StandardScaler()
x_train_scled=scaler.fit_transform(X_train)
x_test_scled=scaler.transform(X_test)

In [14]:
# KNN
scores=[]
for n in [2,3,5,7,9,11]:
    knn=KNeighborsClassifier(n_neighbors=n,metric='l1')
    knn.fit(x_train_scled,y_train)
    y_pred=knn.predict(X_test)
    y_pred_proba=knn.predict_proba(X_test)
    # print(accuracy_score(y_test,y_pred))
    scores.append([n,accuracy_score(y_test,y_pred)])
scores_df=pd.DataFrame(scores,columns=['n_neighbors','Accuracy'])
scores_df.sort_values(by='Accuracy',ascending=False,inplace=True)
scores_df.head()


,n_neighbors,Accuracy
5,11,0.424103
4,9,0.352308
3,7,0.342564
0,2,0.317436
2,5,0.310256


In [15]:
# Gusian Naviae bayes
gnb=GaussianNB()
gnb.fit(X_train,y_train)
y_pred=knn.predict(X_test)
y_pred_proba=knn.predict_proba(X_test)
print(accuracy_score(y_test,y_pred))

0.4241025641025641


In [16]:
# support vector - radial
import numpy as np
scores=[]
c=np.linspace(0.01,5,2)
gama=np.linspace(0.01,5,2)
print(len(c))
# for i in range(1,19):
for i in c:
    for g in gama: 
        svm=SVC(kernel='linear',C=i,gamma=g,probability=True)
        svm.fit(x_train_scled,y_train) 
        y_pred=svm.predict(x_test_scled)
        scores.append([i,g,accuracy_score(y_test,y_pred)])
scores_df=pd.DataFrame(scores,columns=['c','g','Accuracy'])
scores_df.sort_values(by='Accuracy',ascending=False,inplace=True)
scores_df.head()

2


,c,g,Accuracy
0,0.01,0.01,0.537436
1,0.01,5.00,0.537436
2,5.00,0.01,0.535897
3,5.00,5.00,0.535897


In [17]:
# decision tree Classifer
import numpy as np
scores=[]
depth=[2,3,4,5,6,None]
min_sample=[2,10,20,50,75]
min_leaf=[1,10,20,50,75]

for d in depth: 
    for s in min_sample: 
        for l in min_leaf: 
            Dtree= DecisionTreeClassifier(random_state=25,max_depth=d,min_samples_leaf=l,min_samples_split=s)
            Dtree.fit(X_train,y_train) 
            y_pred=Dtree.predict(X_test)
            y_pred_proba=Dtree.predict_proba(X_test)
            scores.append([d,s,l,accuracy_score(y_test,y_pred),log_loss(y_test,y_pred_proba)],)
scores_df=pd.DataFrame(scores,columns=['depth','min_sample','min_leaf','Accuracy','log_loss'])
scores_df.sort_values(by='Accuracy',ascending=False,inplace=True)
scores_df.head()

,depth,min_sample,min_leaf,Accuracy,log_loss
125,NaN,2,1,0.583590,15.008947
130,NaN,10,1,0.547692,11.026023
103,6.0,2,50,0.542051,1.338686
113,6.0,20,50,0.542051,1.338686
108,6.0,10,50,0.542051,1.338686
